In [1]:
###############################
##### importing libraries #####
###############################

import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset   

torch.backends.cudnn.benchmark=True

In [2]:
num_clients = 20
num_selected = 6
num_rounds = 50
epochs = 6
batch_size = 32

In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)), # what is this?
])

In [5]:
df_train = datasets.CIFAR10('../data', train=True, download=True, transform=transform_train)

100%|██████████| 170M/170M [00:05<00:00, 30.2MB/s] 


Extracting ../data/cifar-10-python.tar.gz to ../data


In [ ]:
# split data for a number of num_clients
df_train_split = torch.utils.data.random_split(df_train, [int(df_train.data.shape[0] / num_clients) for _ in range(num_clients)])


In [18]:
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in df_train_split]

In [19]:
# Normalizing the test images
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [20]:
# Load test images to a test loader
# A test loader in PyTorch is .. ?
#    simply DataLoader configured for testing 

test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data', train=False, transform=transform_test), batch_size=batch_size, shuffle=True)

In [21]:
cfg = {
    'VGG11':[64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__() 

        # Are these a common use in Torch, dividing up features and classifier layers?
        ##   Seems like it, to make it easier for transfer learning, modularity, clarity, and memory optimization (Claude)
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Sequential(
            nn.Linear(512, 512), # input and outputs
            nn.ReLU(True), # in PyTorch, the function is added sequentially
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10),
        )

    def forward(self, X):
        out = self.features(X)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        output = F.log_softmax(out, dim=1)
        return output
    
    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for c in cfg:
            if c == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, c, kernel_size=3, padding=1),
                           nn.BatchNorm2d(c),
                           nn.ReLU(inplace=True)] # Why use inplace? ## Saves memory and some performance consideration(Claude)
                in_channels = c
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)



In [30]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [31]:
def client_update(client_model, optimizer, train_loader, epoch=5):
    """
    This function updates/trains client model on client data
    """
    client_model.train()
    for e in range(epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = client_model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
    return loss.item()

In [ ]:
def server_aggregate(global_model, client_models):
    """
    This function has aggregation method 'mean'
    """
    ### This will take simple mean of the weights of models ###
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

    

In [33]:
def test(global_model, test_loader):
    """This function test the global model on test data and returns test loss and test accuracy """
    global_model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = global_model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = correct / len(test_loader.dataset)

    return test_loss, acc

---

In [34]:
device

device(type='mps')

In [35]:

#### global model ##########
global_model =  VGG('VGG11').to(device)

############## client models ##############
client_models = [ VGG('VGG11').to(device) for _ in range(num_selected)]
for model in client_models:
    model.load_state_dict(global_model.state_dict()) ### initial synchronizing with global model 

############### optimizers ################
opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

In [36]:
loss_train = []
loss_test = []
acc_train = []
acc_test = []

In [37]:
for r in range(num_rounds):
    client_idx = np.random.permutation(num_clients)[:num_selected]
    loss = 0
    for i in tqdm(range(num_selected)):
        loss += client_update(client_models[i], opt[i], train_loader[client_idx[i]], epoch=epochs)

    loss_train.append(loss)
    # server aggregate
    server_aggregate(global_model, client_models)
    
    test_loss, acc = test(global_model, test_loader)
    loss_test.append(test_loss)
    acc_test.append(acc)
    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_selected, test_loss, acc))


100%|██████████| 6/6 [01:17<00:00, 12.87s/it]


0-th round
average train loss 1.38 | test loss 2.28 | test acc: 0.194


100%|██████████| 6/6 [01:10<00:00, 11.68s/it]


1-th round
average train loss 1.83 | test loss 1.45 | test acc: 0.463


100%|██████████| 6/6 [01:10<00:00, 11.78s/it]


...

47-th round
average train loss 0.119 | test loss 0.456 | test acc: 0.879


100%|██████████| 6/6 [02:16<00:00, 22.68s/it]


48-th round
average train loss 0.266 | test loss 0.414 | test acc: 0.884


100%|██████████| 6/6 [02:24<00:00, 24.11s/it]


49-th round
average train loss 0.412 | test loss 0.588 | test acc: 0.855


In [39]:
torch.save(global_model.state_dict(), 'global_model.pt')